In [153]:
import pandas as pd
import yfinance as yf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [154]:
symbol = 'TSLA'
start_date = '2015-01-01'
end_date = '2024-04-07'
data = yf.download(symbol, start_date,end_date)

[*********************100%***********************]  1 of 1 completed


In [155]:
#Calculer les returns savoir si TSLA à augmenter ou baisser
data['returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(-1))

In [156]:
data['direction'] = np.where(data['returns'] > 0,1,0)

In [157]:
# ajout de la volatilité
data['Volatility'] = data['Close'].rolling(window=20).std()

In [158]:
# on a un na à la fin on le fill avec la valeur 0
data.tail()
np.mean(data['Volatility'])


6.84589079433816

In [159]:

data['Volatility'] = data['Volatility'].fillna(6.84)
data['returns'] = data['returns'].fillna(0)
data.head()

,Open,High,Low,Close,Adj Close,Volume,returns,direction,Volatility
Date,,,,,,,,,
2015-01-02,14.858000,14.883333,14.217333,14.620667,14.620667,71466000,0.042950,1,6.84
2015-01-05,14.303333,14.433333,13.810667,14.006000,14.006000,80527500,-0.005648,0,6.84
2015-01-06,14.004000,14.280000,13.614000,14.085333,14.085333,93928500,0.001563,1,6.84
2015-01-07,14.223333,14.318667,13.985333,14.063333,14.063333,44526000,0.001566,1,6.84
2015-01-08,14.187333,14.253333,14.000667,14.041333,14.041333,51637500,0.018981,1,6.84


In [160]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume,returns,direction,Volatility
Date,,,,,,,,,
2024-04-01,176.169998,176.750000,170.210007,175.220001,175.220001,81562100,0.050267,1,5.741573
2024-04-02,164.750000,167.690002,163.429993,166.630005,166.630005,116650600,-0.010448,0,5.086943
2024-04-03,164.020004,168.820007,163.279999,168.380005,168.380005,82950100,-0.016083,0,4.937359
2024-04-04,170.070007,177.190002,168.009995,171.110001,171.110001,123162000,0.036967,1,4.887195
2024-04-05,169.080002,170.860001,160.509995,164.899994,164.899994,141250700,0.000000,0,4.990970


In [161]:
data['direction'].value_counts()
# cuoff de 0.516 car 1039/(974+1039) = 0.516

direction
0    1209
1    1121
Name: count, dtype: int64

In [162]:
# Split en 70 / 30 
train_set, test_set = np.split(data, [int(.70 *len(data))])
y_train = train_set['direction']

x_test = test_set['direction']



c:\Users\alrem\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [163]:
# Instancier le modèle de régression logistique
model = LogisticRegression()

In [164]:
# Entraîner le modèle sur les données d'entraînement
model.fit(train_set, y_train)

LogisticRegression()

In [165]:
y_pred_probs = model.predict_proba(test_set)
print(y_pred_probs)
y_pred_classes = (y_pred_probs[:, 1] > 0.49).astype(int)
print(y_pred_classes)
# faire un merge entre mes prédictions et faire un shift(1)
# Convertir les probabilités prédites en classes
# y_pred_classes = np.argmax(y_pred, axis=1)
# print(y_pred_classes)

[[0.51104387 0.48895613]
 [0.50735092 0.49264908]
 [0.50590765 0.49409235]
 ...
 [0.50939717 0.49060283]
 [0.51395068 0.48604932]
 [0.5159983  0.4840017 ]]
[0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 1 1 0 0 1 1 1 1 1
 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 0 0 1
 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0
 1 0 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1 0
 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1
 1 0 0 1 0 1 1 1 1 0 0 0 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1
 0 1 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 

In [166]:

y_pred_classes_shifted = np.concatenate(([0], y_pred_classes[:-1]))

# Affichez le tableau décalé
print(y_pred_classes_shifted)
# demain 08/04/2024 tesla devrait baisser

[0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 0 1 0 0 0 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 1 1 0 0 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 0 0 1 0 1 0 1 1 0 1 1 1 1 0 0
 1 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 0 0 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 0 1 0 1 1 1 1 0 1 0 1
 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1
 1 1 0 0 1 0 1 1 1 1 0 0 0 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1
 1 0 1 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 1 1 1 0 1 0 1 0 1 0 1 1 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [167]:
# Évaluer les performances du modèle
accuracy = accuracy_score(x_test, y_pred_classes_shifted)
print("Accuracy:", accuracy)

# Afficher un rapport de classification
print("\nClassification Report:")
print(classification_report(x_test, y_pred_classes_shifted))

Accuracy: 0.5293276108726752

Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.62      0.58       368
           1       0.50      0.43      0.46       331

    accuracy                           0.53       699
   macro avg       0.53      0.52      0.52       699
weighted avg       0.53      0.53      0.53       699



In [168]:
# Sélectionner les variables indépendantes et dépendante
X = data[['Volume', 'Adj Close','Volatility']]  # Variables indépendantes
y = data['direction']  # Variable dépendante

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [169]:
# Instancier et entraîner le modèle de régression logistique
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [170]:
# Obtenir les probabilités prédites au lieu des classes prédites
proba_pred = model.predict_proba(X_test)
y_pred_classes = (proba_pred[:, 1] > 0.49).astype(int)
print(y_pred_classes)


[1 0 1 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0
 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 0 0 1 1 0 1 1 1 0 

In [171]:

y_pred_classes_shifted = np.concatenate(([0], y_pred_classes[:-1]))

# Affichez le tableau décalé
print(y_pred_classes_shifted)

[0 1 0 1 0 0 1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 0 0 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 0 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 0 1 1 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1
 0 0 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0
 1 1 1 1 0 0 1 1 0 1 1 1 

In [172]:
# Évaluer les performances du modèle
accuracy = accuracy_score(y_test, y_pred_classes_shifted)
print("Accuracy:", accuracy)

# Afficher un rapport de classification
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes_shifted))

Accuracy: 0.48068669527896996

Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.14      0.23       382
           1       0.46      0.89      0.61       317

    accuracy                           0.48       699
   macro avg       0.53      0.52      0.42       699
weighted avg       0.54      0.48      0.40       699

